In [1]:
from  scipy import ndimage
import math
import numpy as np
import argparse
import cv2
from math import atan2, pi
import string
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
from keras.models import Sequential, load_model
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
from nltk.tag import pos_tag
import json
from fuzzywuzzy import fuzz

c:\users\sachin\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ModuleNotFoundError: No module named 'fuzzywuzzy'

In [ ]:
imageFilePath =r'C:\Users\sachin\Desktop\Self-Declaration.jpg'

KERAS_Alphabet_model_File_Path = r'C:\Users\sachin\Desktop\Keras_Alphabet_Model.h5'
Alphabet_Mapping_List = list(string.ascii_uppercase)
kernel = np.ones((3,3),np.uint8)
index = 0
global counts,X,Y,W,H

In [4]:
try:
    KERAS_Alphabet_Model = load_model(KERAS_Alphabet_model_File_Path)
except:
    KERAS_Model = load_model(KERAS_model_File_Path)

In [1]:
def Get_Processed_Image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(gray_image,(5,5),0)
    ret3,Otsu_Threshold_GaussianBlur_Image = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    Otsu_Threshold_GaussianBlur_Image = cv2.bitwise_not(Otsu_Threshold_GaussianBlur_Image)

    closing = cv2.morphologyEx(Otsu_Threshold_GaussianBlur_Image, cv2.MORPH_CLOSE, kernel)
    #cv2.imwrite(r'C:\Users\sachin13390\Desktop\closing.jpg',closing)

    return closing



def Get_Dilated_Image(image, number):
    kernel = np.ones((5,number),np.uint8)
    img_dilation = cv2.dilate(image, kernel, iterations=3)
    #cv2.imwrite(r'C:\Users\sachin13390\Desktop\img_dilation_'+str(index)+'.jpg',img_dilation)

    return img_dilation
        

def Get_Countours(input_Image):
    temp_image, contours, hierarchy = cv2.findContours(input_Image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return contours


def Sort_Countours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0

    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True

    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1

    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b:b[1][i], reverse=reverse))

    # return the list of sorted contours and bounding boxes
    return (cnts)

def Draw_Contours(Image, Contours):
    for cnt in Contours:
        x,y,w,h = cv2.boundingRect(cnt)
        #print (str(x) + " - " + str(y)+ " - " + str(w) + " - " + str(h))
        cv2.rectangle(Image,(x,y),(x+w,y+h),(0,255,255),1)
    cv2.imwrite(r'C:\Users\sachin13390\Desktop\croppedrect.jpg',Image)   




def getNewResizedImage(input_Image, Image_Size):
    height,width = input_Image.shape
    #print (height, width)

    if width > height:
        aspect_Ratio = (float)(width/height)
        width = 20
        height = round(width/aspect_Ratio)
    else:
        aspect_Ratio = (float)(height/width)
        height = 20
        width = round(height/aspect_Ratio)
        
    input_Image = cv2.resize(input_Image, (width,height), interpolation = cv2.INTER_AREA )
    
    height,width = input_Image.shape
    
    number_Of_Column_To_Add = 28-width
    temp_Column = np.zeros( (height , int(number_Of_Column_To_Add/2)), dtype = np.uint8)
    input_Image = np.append(temp_Column, input_Image, axis=1)
    input_Image = np.append(input_Image, temp_Column, axis=1)


    height,width = input_Image.shape

    number_Of_Row_To_Add = 28-height
    temp_Row= np.zeros( (int(number_Of_Row_To_Add/2) , width ), dtype = np.uint8)
    input_Image = np.concatenate((temp_Row,input_Image))
    input_Image = np.concatenate((input_Image,temp_Row))

    return cv2.resize(input_Image, (Image_Size,Image_Size), interpolation = cv2.INTER_AREA )


def Get_Average_Space_From_Image_Line(contours):
    total_space = 0
    number_Of_Character = 0
    last_Point = -1
    avg_Space = 0
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        #print (x,y,w,h)

        if w > 30 and h > 30 and x > 0 and y > 0:
            if last_Point == -1:
                last_Point = 0
            else:
                total_space = total_space + ( x - last_Point )
            last_Point = x + w

            number_Of_Character = number_Of_Character + 1

    avg_Space = int (total_space/number_Of_Character)

    total_space = 0
    last_Point = -1
    number_Of_Character = 0

##    for cnt in contours:
##        x,y,w,h = cv2.boundingRect(cnt)
##
##        if w > 30 and h > 30 and x > 0 and y > 0:
##            if last_Point == -1:
##                last_Point = 0
##            else:
##                space = ( (x + w) - last_Point )
##                if space < avg_Space:
##                    total_space = total_space + space
##                    number_Of_Character = number_Of_Character + 1
##            last_Point = x + w
##            
##    avg_Space = int (total_space/number_Of_Character)

    return avg_Space







def Get_Text_From_Image(Image, contours):
    global count,X,Y,W,H, list_Character_Positions
    KERAS_Alphabet_Prediction = ''

    total_space = 0
    number_Of_Character = 0
    last_Point = -1

    #avg_Space = Get_Average_Space_From_Image_Line(contours)
    last_Point = -1
    total_space = 0
    #print (avg_Space)
    
    Word_Dilated_Image = Get_Dilated_Image(Image,20)
    Word_Contours = Get_Countours(Word_Dilated_Image)
    Word_Contours = Sort_Countours(Word_Contours,"left-to-right")
    last_Word_Contour_Index = 0
    Word_X, Word_Y, Word_W, Word_H = cv2.boundingRect(Word_Contours[last_Word_Contour_Index])
    last_Word_Contour_Max_X_Range = Word_X + Word_W
    # i.e X + W
            

    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        give_Space = False
        #print (x,y,w,h)

        # Reject if Contour is not of desired size (too small)
        if w > 30 and h > 30 and x > 0 and y > 0:

#             ## Spacing based on average space between character
#             if last_Point == -1:
#                 last_Point = 0
#             else:
#                 total_space = ( x - last_Point )

#             last_Point = x + w

#             if total_space > (avg_Space*1.3):
#                 give_Space = True
            
            ## Spacing based on word formation
            if x > last_Word_Contour_Max_X_Range:
                give_Space = True
                last_Word_Contour_Index = last_Word_Contour_Index + 1
                Word_X, Word_Y, Word_W, Word_H = cv2.boundingRect(Word_Contours[last_Word_Contour_Index])
                last_Word_Contour_Max_X_Range = Word_X + Word_W
                # i.e X + W
            
    
                
            if give_Space == True:
                KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + " "
                list_Character_Positions.append((-1,-1,-1,-1," "))
                

            resize_image = getNewResizedImage(Image[y:y+h, x:x+w] , 28)
            path = r'C:\Users\sachin13390\Desktop\Images\\' + str(count)+'.png'
            cv2.imwrite(path,resize_image)
            resize_image = resize_image.flatten()
            count = count + 1

            temp_Index = int(KERAS_Alphabet_Model.predict_classes(resize_image.reshape(1,784)/255.0)[0])
            alphabet_probability = (KERAS_Alphabet_Model.predict_proba(resize_image.reshape(1,784)/255.0))
            sort_alphabet_probability = -np.sort(-alphabet_probability)

            KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + Alphabet_Mapping_List[int(temp_Index)]

            list_Character_Positions.append((x+X,y+Y,w,h,str(Alphabet_Mapping_List[int(temp_Index)])))

##            if sort_alphabet_probability[0,0] > 0.99:
##                KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + Alphabet_Mapping_List[int(temp_Index)]
##            else:
##                alternate_Probable_Alphabet = Alphabet_Mapping_List[int(np.where(alphabet_probability == sort_alphabet_probability[0,1])[1][0])]
##                KERAS_Alphabet_Prediction = KERAS_Alphabet_Prediction + "[" + Alphabet_Mapping_List[int(temp_Index)] + "," + alternate_Probable_Alphabet + "]"

    list_Character_Positions.append((-1,-1,-1,-1," "))

    return KERAS_Alphabet_Prediction

In [31]:
image = cv2.imread(imageFilePath)
image = Get_Processed_Image(image)
image_With_Lines = Get_Dilated_Image(image, 60)

contours = Get_Countours(image_With_Lines)
contours = Sort_Countours(contours, "top-to-bottom")

Draw_Contours(cv2.imread(imageFilePath),contours)
Predicted_Text = ''
temp_Index = ''
global list_Character_Positions
X=Y=W=H=0
count = 20
list_Character_Positions = []

In [32]:
counts = 0
index = 0
Predicted_Text = ''
for line_Area in contours:
    counts = counts + 1
    x,y,w,h = cv2.boundingRect(line_Area)
    X,Y,W,H = x,y,w,h
    line_Image = image[y:y+h, x:x+w]
    path = r'C:\Users\sachin13390\Desktop\Images\\' + str(counts)+'.png'
    cv2.imwrite(path,line_Image)

    line_Contours = Get_Countours(line_Image)
    line_Contours = Sort_Countours(line_Contours,"left-to-right")
    #Draw_Contours(line_Image,line_Contours)
    
    text = (Get_Text_From_Image(line_Image, line_Contours))
    #print (text)

    Predicted_Text = Predicted_Text + " " + text


print('KERAS_Alphabet_Predict \n' + Predicted_Text)

KERAS_Alphabet_Predict 
 MY NAME IS HEMANT SMELAR I AM WORKING JN MASTEK FROM LAST SIX TEARS CURRENTLY WORKING POR JNDIAN AND UR GOVERNMENT PROIECT


In [33]:
tagged_sent = pos_tag(Predicted_Text.title().split())

propernouns = [word for word,pos in tagged_sent if pos == 'NNP']

for names in propernouns:
    print (names)

Hemant
Smelar
Jn
Mastek
Six
Tears
Currently
Working
Por
Jndian
Ur
Government
Proiect


In [34]:
image = cv2.imread(imageFilePath)
string_Output = ""
for char in list_Character_Positions:
    string_Output = string_Output + char[4]
    #print (char[0],char[1],char[2],char[3],char[4])
    x,y,w,h = char[0],char[1],char[2],char[3]
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,0),1)
cv2.imwrite(r'C:\Users\sachin13390\Desktop\Final.jpg',image)

json_list_Character_Positions = json.dumps(list_Character_Positions)
print (json_list_Character_Positions)

[[378, 187, 81, 61, "M"], [480, 188, 59, 69, "Y"], [-1, -1, -1, -1, " "], [630, 191, 59, 66, "N"], [718, 184, 52, 67, "A"], [792, 192, 84, 59, "M"], [904, 187, 49, 68, "E"], [-1, -1, -1, -1, " "], [1026, 178, 48, 72, "I"], [1095, 183, 47, 66, "S"], [-1, -1, -1, -1, " "], [1245, 175, 53, 72, "H"], [1318, 176, 50, 73, "E"], [1385, 175, 99, 66, "M"], [1523, 171, 78, 73, "A"], [1617, 185, 72, 66, "N"], [1705, 186, 77, 62, "T"], [-1, -1, -1, -1, " "], [1890, 188, 81, 63, "S"], [1984, 199, 62, 65, "M"], [2073, 192, 51, 66, "E"], [2158, 198, 63, 65, "L"], [2237, 191, 62, 74, "A"], [2320, 181, 94, 76, "R"], [-1, -1, -1, -1, " "], [363, 360, 67, 67, "I"], [-1, -1, -1, -1, " "], [502, 370, 71, 70, "A"], [592, 377, 94, 67, "M"], [-1, -1, -1, -1, " "], [827, 383, 92, 62, "W"], [954, 390, 46, 53, "O"], [1029, 378, 74, 67, "R"], [1123, 381, 72, 63, "K"], [1213, 376, 61, 70, "I"], [1306, 385, 77, 62, "N"], [1417, 385, 57, 72, "G"], [-1, -1, -1, -1, " "], [1552, 377, 63, 67, "J"], [1658, 388, 73, 69, 

In [35]:
def Get_Words_To_Hide_In_List(words_To_Hide):
    words_To_Hide = words_To_Hide.upper().strip().replace('  ',' ')
    
    while '  ' in words_To_Hide:
        words_To_Hide =  words_To_Hide.replace('  ',' ')
    list_Words_To_Hide = words_To_Hide.split(" ")
    
    return list_Words_To_Hide

#words_To_Hide = input("Enter word to hide ")
words_To_Hide = "  Hemant  MASTEK  Project    Shelar    Working  kiosk"

list_Words_To_Hide = Get_Words_To_Hide_In_List(words_To_Hide)
print (list_Words_To_Hide)

['HEMANT', 'MASTEK', 'PROJECT', 'SHELAR', 'WORKING', 'KIOSK']


In [36]:
global word_Matching_Thresh_Value
word_Matching_Thresh_Value = 80

def Get_Similar_List_Of_Words_To_Hide(list_Words_To_Hide):
    list_Similar_Words = []
    list_Predicted_Text_Words = string_Output.split(" ")
    
    for word_To_Hide in list_Words_To_Hide:
        
        for predicted_Word in list_Predicted_Text_Words:
            
            word_Probability = fuzz.ratio(word_To_Hide,predicted_Word)
            
            if word_Probability > word_Matching_Thresh_Value:
                list_Similar_Words.append(predicted_Word)
                #print (word_To_Hide,predicted_Word)
                
    return list_Similar_Words

In [37]:
list_Similar_Words = Get_Similar_List_Of_Words_To_Hide(list_Words_To_Hide)

for similar_Word in list_Similar_Words:
    if not similar_Word in list_Words_To_Hide:
        list_Words_To_Hide.append(similar_Word)

print (list_Words_To_Hide)

['HEMANT', 'MASTEK', 'PROJECT', 'SHELAR', 'WORKING', 'KIOSK', 'PROIECT', 'SMELAR']


In [38]:
image = cv2.imread(imageFilePath)

for word in list_Words_To_Hide:

    count_Of_Word = string_Output.count(word)
    
    print (word,count_Of_Word)
    word_index = -1
    char_index = 0
    
    for index in range(0,count_Of_Word):
        word_index = string_Output.find(word, word_index + 1)
        Min_X = Min_Y = 10000
        Max_W = Max_H = 0
        
        for i in range (word_index, word_index + len(word)):
            char = list_Character_Positions[i]
            x,y,w,h = char[0],char[1],char[2],char[3]
            
            if Min_X > x and x > -1:
                Min_X = x
            if Min_Y > y and y > -1:
                Min_Y = y
            if Max_W < (x + w):
                Max_W = (x + w)
            if Max_H < (y + h):
                Max_H = (y + h)
                
            char_index = char_index + 1
        cv2.rectangle(image,(Min_X,Min_Y),(Max_W,Max_H),(0,0,0),-1)

cv2.imwrite(r'C:\Users\sachin13390\Desktop\Final.jpg',image)
    

HEMANT 1
MASTEK 1
PROJECT 0
SHELAR 0
WORKING 2
KIOSK 0
PROIECT 1
SMELAR 1


True